<a href="https://colab.research.google.com/github/saisrikanthmadugula/rkAMM-Simulation-Framework/blob/main/Generate_Loan_Data_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sys

# --- CONSTANTS ---
REPAY_BUMP = 3
DEFAULT_PENALTY = -15
MAX_COLLATERAL = 0.8 # Max required collateral is 80%
MIN_P_D = 0.01       # 1% (for 100-rep borrower)
MAX_P_D = 0.80       # 80% (for 0-rep borrower)

# --- AGENT DEFINITIONS ---

class RiskAssessmentAgent:
    """
    This agent is STATEFUL. It maintains a memory of each
    borrower's reputation and updates it based on loan outcomes.
    It also dynamically determines the required collateral.
    """
    def __init__(self, initial_borrowers):
        self.borrower_reputations = {
            borrower_id: np.clip(np.random.normal(loc=45, scale=10), 10, 80)
            for borrower_id in initial_borrowers
        }

    def get_reputation(self, borrower_id):
        return self.borrower_reputations.get(borrower_id)

    def assess_loan_terms(self, borrower_id):
        """
        Generates p_d and C based on the borrower's current reputation.
        """
        reputation = self.get_reputation(borrower_id)
        p_d = MAX_P_D - (reputation / 100) * (MAX_P_D - MIN_P_D)
        collateral_level = MAX_COLLATERAL * (1.0 - (reputation / 100))
        return np.clip(p_d, MIN_P_D, MAX_P_D), np.clip(collateral_level, 0, MAX_COLLATERAL)

    def update_reputation(self, borrower_id, outcome):
        """
        The feedback loop. Updates reputation based on loan outcome.
        """
        if outcome == "Repaid":
            self.borrower_reputations[borrower_id] += REPAY_BUMP
        else: # Default
            self.borrower_reputations[borrower_id] += DEFAULT_PENALTY

        self.borrower_reputations[borrower_id] = np.clip(
            self.borrower_reputations[borrower_id], 0, 100
        )

class PricingAgent:
    """
    Uses the Reverse Kelly AMM logic from Esteva et al. (2023)
    to calculate the required premium for a loan.
    """
    def __init__(self):
        pass

    def calculate_premium(self, p_d, collateral_level, loan_amount=1.0):
        L = loan_amount * (1.0 - collateral_level)
        if L <= 0 or p_d <= 0:
            return 0.0, L
        if p_d >= 1.0:
            return np.inf, L
        premium = (L * p_d) / (1.0 - p_d)
        return premium, L

class BlockchainEnforcementAgent:
    """
    Simulates the execution of the loan on the blockchain.
    """
    def __init__(self):
        pass

    def execute_loan(self, p_d):
        return "Repaid" if np.random.rand() >= p_d else "Default"

# --- SIMULATION ENGINE ---

class Simulation:
    """
    Runs the dynamic simulation with a fixed pool of borrowers
    taking out multiple loans.
    """
    def __init__(self, n_borrowers, loans_per_borrower):
        self.n_borrowers = n_borrowers
        self.loans_per_borrower = loans_per_borrower
        self.borrower_ids = list(range(n_borrowers))

        self.risk_agent = RiskAssessmentAgent(self.borrower_ids)
        self.pricing_agent = PricingAgent()
        self.blockchain_agent = BlockchainEnforcementAgent()

        self.results = []
        self.total_loan_id = 0

    def run(self):
        """
        Run the simulation loop.
        """
        print("Running simulation...")
        for loan_num in range(self.loans_per_borrower):
            # Print progress
            sys.stdout.write(f"\rProcessing loan round {loan_num + 1}/{self.loans_per_borrower}")
            sys.stdout.flush()

            np.random.shuffle(self.borrower_ids)

            for borrower_id in self.borrower_ids:
                current_rep = self.risk_agent.get_reputation(borrower_id)
                p_d, collateral_level = self.risk_agent.assess_loan_terms(borrower_id)

                premium, loss_given_default = self.pricing_agent.calculate_premium(
                    p_d, collateral_level
                )

                outcome = self.blockchain_agent.execute_loan(p_d)

                self.risk_agent.update_reputation(borrower_id, outcome)

                self.results.append({
                    "total_loan_id": self.total_loan_id,
                    "loan_round": loan_num,
                    "borrower_id": borrower_id,
                    "initial_reputation": current_rep,
                    "p_d": p_d,
                    "collateral_level": collateral_level,
                    "premium": premium,
                    "loss_given_default": loss_given_default,
                    "outcome": outcome,
                    "final_reputation": self.risk_agent.get_reputation(borrower_id)
                })
                self.total_loan_id += 1

        print("\nSimulation complete.")
        return pd.DataFrame(self.results)

# --- SCRIPT EXECUTION & SAVING ---

if __name__ == "__main__":

    N_BORROWERS = 500
    LOANS_PER_BORROWER = 20
    OUTPUT_FILE = "all_10000_loans.csv"

    print(f"Generating {N_BORROWERS * LOANS_PER_BORROWER} loan records...")

    sim = Simulation(n_borrowers=N_BORROWERS, loans_per_borrower=LOANS_PER_BORROWER)
    df_results = sim.run()

    # Save the entire DataFrame to a CSV file
    try:
        df_results.to_csv(OUTPUT_FILE, index=False)
        print(f"\nSuccessfully saved all 10,000 loan records to:")
        print(f"{OUTPUT_FILE}")

        print("\n--- Data Snapshot (First 5 loans) ---")
        print(df_results.head())

    except Exception as e:
        print(f"\nError saving file: {e}")